## Imports

In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
bracket_training = pd.read_csv('bracket_training_with_names.csv')
bracket_training.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,UConn,Arizona,James Madison,Tennessee,UConn,Tennessee,Tennessee
1,1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,UConn,Baylor,Kentucky,Kansas,UConn,Kentucky,UConn
2,2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,UConn,Baylor,Houston,Purdue,UConn,Purdue,Purdue
3,3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 10:28:56 -0400,Iowa St.,Saint Mary's,NC State,Purdue,Iowa St.,NC State,NC State
4,4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,2022-03-16,2828017,2024-03-20 20:14:52 -0400,Auburn,North Carolina,Marquette,Creighton,North Carolina,Marquette,North Carolina


In [22]:
bracket_test = pd.read_csv('bracket_test_with_names.csv')
bracket_test['SemifinalWinner_East_West'] = ""
bracket_test['SemifinalWinner_South_Midwest'] = ""
bracket_test.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest
0,0,73662,919.0,27539,35.7225,-78.8408,560.0,RALEIGH - DURHAM (FAYETVLLE),3/29/20,2074118,3/19/24 18:50,UConn,North Carolina,Houston,Purdue,,
1,1,6679,360.0,97206,45.4840,-122.5973,820.0,"PORTLAND, OR",4/2/24,2692634,3/20/24 16:56,UConn,North Carolina,Duke,Kansas,,
2,2,63024,270.0,42754,37.4603,-86.3249,529.0,LOUISVILLE,12/8/21,1252684,3/18/24 15:13,Iowa St.,Arizona,Kentucky,Creighton,,
3,3,60371,206.0,98178,47.4924,-122.2359,819.0,SEATTLE - TACOMA,3/22/23,1950205,3/19/24 15:21,UConn,North Carolina,Houston,Purdue,,
4,4,18415,717.0,19038,40.1096,-75.1550,504.0,PHILADELPHIA,2/20/24,2756293,3/20/24 18:40,UConn,North Carolina,Marquette,Creighton,,


In [4]:
bracket_training.shape

(130002, 18)

In [5]:
bracket_test.shape

(14445, 16)

## Semifinal 1 Predictions

In [12]:
bracket_training_simple = bracket_training[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId']]
bracket_training_simple['CustomerDMACode'] = bracket_training_simple['CustomerDMACode'].astype(str).str[:-2]
bracket_training_simple.set_index('BracketEntryId', inplace=True)
bracket_training_simple.head()

/var/folders/66/kjdgfk_530v31znwf93fdr6h0000gn/T/ipykernel_14601/3645458893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_training_simple['CustomerDMACode'] = bracket_training_simple['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_East,RegionWinner_West,SemifinalWinner_East_West
BracketEntryId,,,,
1723503,698,UConn,Arizona,UConn
963479,563,UConn,Baylor,UConn
810038,511,UConn,Baylor,UConn
3384825,641,Iowa St.,Saint Mary's,Iowa St.
2828017,514,Auburn,North Carolina,North Carolina


In [13]:
bracket_test_simple = bracket_test[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId']]
bracket_test_simple['CustomerDMACode'] = bracket_test_simple['CustomerDMACode'].astype(str).str[:-2]
bracket_test_simple.set_index('BracketEntryId', inplace=True)
bracket_test_simple.head()

/var/folders/66/kjdgfk_530v31znwf93fdr6h0000gn/T/ipykernel_14601/2724997793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_simple['CustomerDMACode'] = bracket_test_simple['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_East,RegionWinner_West,SemifinalWinner_East_West
BracketEntryId,,,,
2074118,560,UConn,North Carolina,
2692634,820,UConn,North Carolina,
1252684,529,Iowa St.,Arizona,
1950205,819,UConn,North Carolina,
2756293,504,UConn,North Carolina,


In [14]:
# Define training data (from bracket_training_simple)
X_train = bracket_training_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']]
y_train = bracket_training_simple['SemifinalWinner_East_West']

# Define test data (from bracket_test_simple)
X_test = bracket_test_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']]

# Define categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']

# Initialize and train the model
model = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, cat_features=cat_features, verbose=2)
model.fit(X_train, y_train)

0:	learn: 2.0076355	total: 2.94s	remaining: 9m 44s
2:	learn: 1.5984883	total: 8.7s	remaining: 9m 31s
4:	learn: 1.3645478	total: 14.6s	remaining: 9m 28s
6:	learn: 1.2045209	total: 20.7s	remaining: 9m 29s
8:	learn: 1.0937118	total: 26.8s	remaining: 9m 28s
10:	learn: 1.0082231	total: 32.9s	remaining: 9m 25s
12:	learn: 0.9415755	total: 39.2s	remaining: 9m 23s
14:	learn: 0.8876695	total: 45.6s	remaining: 9m 22s
16:	learn: 0.8447867	total: 52s	remaining: 9m 19s
18:	learn: 0.8114042	total: 58.4s	remaining: 9m 16s
20:	learn: 0.7827070	total: 1m 4s	remaining: 9m 13s
22:	learn: 0.7600668	total: 1m 11s	remaining: 9m 12s
24:	learn: 0.7404337	total: 1m 18s	remaining: 9m 11s
26:	learn: 0.7224135	total: 1m 25s	remaining: 9m 10s
28:	learn: 0.7100570	total: 1m 33s	remaining: 9m 12s
30:	learn: 0.7000705	total: 1m 41s	remaining: 9m 13s
32:	learn: 0.6922601	total: 1m 48s	remaining: 9m 10s
34:	learn: 0.6839389	total: 1m 56s	remaining: 9m 8s
36:	learn: 0.6767425	total: 2m 4s	remaining: 9m 9s
38:	learn: 0.67

ValueError: 2

In [16]:
bracket_test_simple['SemifinalWinner_East_West'] = model.predict(X_test).flatten()
bracket_test_simple

/var/folders/66/kjdgfk_530v31znwf93fdr6h0000gn/T/ipykernel_14601/87650167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_simple['SemifinalWinner_East_West'] = model.predict(X_test).flatten()


,CustomerDMACode,RegionWinner_East,RegionWinner_West,SemifinalWinner_East_West
BracketEntryId,,,,
2074118,560,UConn,North Carolina,North Carolina
2692634,820,UConn,North Carolina,UConn
1252684,529,Iowa St.,Arizona,Iowa St.
1950205,819,UConn,North Carolina,UConn
2756293,504,UConn,North Carolina,UConn
...,...,...,...,...
2085292,789,UConn,Arizona,Arizona
615949,516,UConn,North Carolina,UConn
13727,862,BYU,Arizona,Arizona


## SemiFinal 2 Predictions

In [20]:
bracket_training_semifinal2 = bracket_training[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_South_Midwest', 'BracketEntryId']]
bracket_training_semifinal2['CustomerDMACode'] = bracket_training_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_training_semifinal2.set_index('BracketEntryId', inplace=True)
bracket_training_semifinal2.head()

/var/folders/66/kjdgfk_530v31znwf93fdr6h0000gn/T/ipykernel_14601/638671133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_training_semifinal2['CustomerDMACode'] = bracket_training_semifinal2['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_South_Midwest
BracketEntryId,,,,
1723503,698,James Madison,Tennessee,Tennessee
963479,563,Kentucky,Kansas,Kentucky
810038,511,Houston,Purdue,Purdue
3384825,641,NC State,Purdue,NC State
2828017,514,Marquette,Creighton,Marquette


In [23]:
bracket_test_semifinal2 = bracket_test[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_South_Midwest', 'BracketEntryId']]
bracket_test_semifinal2['CustomerDMACode'] = bracket_test_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_test_semifinal2.set_index('BracketEntryId', inplace=True)
bracket_test_semifinal2.head()

/var/folders/66/kjdgfk_530v31znwf93fdr6h0000gn/T/ipykernel_14601/3465230186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_semifinal2['CustomerDMACode'] = bracket_test_semifinal2['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_South_Midwest
BracketEntryId,,,,
2074118,560,Houston,Purdue,
2692634,820,Duke,Kansas,
1252684,529,Kentucky,Creighton,
1950205,819,Houston,Purdue,
2756293,504,Marquette,Creighton,


In [25]:
# Define training data (from bracket_training_simple)
X_train = bracket_training_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest',]]
y_train = bracket_training_semifinal2['SemifinalWinner_South_Midwest']

# Define test data (from bracket_test_simple)
X_test = bracket_test_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest',]]

# Define categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest',]

# Initialize and train the model
model = CatBoostClassifier(iterations=150, depth=6, learning_rate=0.1, cat_features=cat_features, verbose=2)
model.fit(X_train, y_train)

bracket_test_semifinal2['SemifinalWinner_East_West'] = model.predict(X_test).flatten()
bracket_test_semifinal2

0:	learn: 2.1810939	total: 1.97s	remaining: 4m 53s
2:	learn: 1.7655889	total: 5.53s	remaining: 4m 30s
4:	learn: 1.5272732	total: 9.06s	remaining: 4m 22s
6:	learn: 1.3743278	total: 12.6s	remaining: 4m 17s
8:	learn: 1.2415548	total: 16.2s	remaining: 4m 14s
10:	learn: 1.1442270	total: 20s	remaining: 4m 12s
12:	learn: 1.0656147	total: 23.7s	remaining: 4m 9s
14:	learn: 1.0027099	total: 27.6s	remaining: 4m 8s
16:	learn: 0.9514751	total: 32s	remaining: 4m 10s
18:	learn: 0.9056281	total: 37.5s	remaining: 4m 18s
